# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [139]:
import sparker
import random

In [140]:
import pandas as pd

In [141]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [142]:
# Profiles contained in the first dataset
profiles = sparker.CSVWrapper.load_profiles('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/sample1_sample2.csv',
                                            start_id_from=0,
                                            separator=",", header=True,
                                            real_id_field = "id",
                                            source_id=0)

In [143]:
# Profiles contained in the first dataset
# profiles = sparker.JSONWrapper.load_profiles('/Final_Project/sample/Df2.json', real_id_field = "id")

Let's visualize a profile to check if they are correctly loaded

In [144]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'name', 'value': 'Omkar   jori'}, {'key': 'fathername', 'value': 'Nathu jori'}, {'key': 'mothername', 'value': 'Aruna jori'}, {'key': 'dateofbirth', 'value': '1999-01-13'}, {'key': 'location', 'value': 'kothrud'}, {'key': 'city', 'value': 'Pune'}, {'key': 'state', 'value': 'Maharastra'}, {'key': 'country', 'value': 'India'}, {'key': 'pincode', 'value': '411038'}, {'key': 'gender', 'value': 'M'}, {'key': 'qualification', 'value': 'BE in mechnical'}, {'key': 'hobby', 'value': 'swimming'}], 'original_id': '1', 'source_id': 0}


In [145]:
profiles.take(5)
profiles.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{name, Omkar   j...|          1|         0|        0|
|[{name, Mr Gupta}...|          2|         1|        0|
|[{name, Amruta  A...|          3|         2|        0|
|[{name, varun jha...|          4|         3|        0|
|[{name, Aditi Pat...|          5|         4|        0|
|[{name, Bhumik Pa...|          6|         5|        0|
|[{name, Miss Amru...|          7|         6|        0|
|[{name, Omkar  na...|          8|         7|        0|
|[{name, Rohit gup...|          9|         8|        0|
|[{name, Argade Am...|         10|         9|        0|
|[{name, varun jha...|         11|        10|        0|
|[{name, Patil Adi...|         12|        11|        0|
|[{name, Bhumika H...|         13|        12|        0|
|[{name, Mrs Deshm...|         14|        13|        0|
|[{name, Amruta Ar...|         15|        14|   

Extract the max id (it will be used later)

In [146]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

In [147]:
max_profile_id

22


### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [148]:
# # Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# # respectively the id of profiles of the first dataset and the id of profiles of the second dataset
# gt = sparker.CSVWrapper.load_groundtruth('../datasets/dirty/cora/groundtruth.csv', 'id1', 'id2')

In [149]:
# # Converts the groundtruth by replacing original IDs with those given by Spark
# new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [150]:
# # We can explore some pairs
# random.sample(new_gt, 2)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [151]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 591


Let's continue by using the standard token blocking

In [152]:
# blocks = sparker.Blocking.create_blocks(profiles)
# print("Number of blocks",blocks.count())

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [153]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [154]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [155]:
# recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

# print("Recall", recall)
# print("Precision", precision)
# print("Number of comparisons", cmp_n)

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [156]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [157]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [158]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 74


### Reciprocal Weighted Node Pruning

In [159]:
# results = sparker.WNP.wnp(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           #groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.AND
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# # print("Recall", num_matches/len(new_gt))
# # print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Weighted Edge Pruning

In [160]:
# results = sparker.WEP.wep(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           #groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# # print("Recall", num_matches/len(new_gt))
# # print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Cardinality Node Pruning

In [161]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.OR
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Reciprocal Cardinality Node Pruning

In [162]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.AND
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Cardinality Edge Pruning

In [163]:
# results = sparker.CEP.cep(
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           #groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# # print("Recall", num_matches/len(new_gt))
# # print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [164]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 4, 26),
 (0, 7, 87),
 (0, 9, 35),
 (0, 11, 26),
 (0, 13, 24),
 (0, 2, 29),
 (0, 21, 7),
 (7, 8, 18),
 (7, 9, 40),
 (7, 11, 25)]

In [165]:
edges_df = pd.DataFrame(edges.collect())

In [166]:
edges_df

0   1   2
0    0   7  87
1    0   2  29
2    0   9  35
3    0  11  26
4    0  13  24
..  ..  ..  ..
69  18  20  13
70  18  22  12
71  18  19  40
72  20  22  17
73  19  21   7

[74 rows x 3 columns]

In [167]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          7|            0|[{0, 2, 29}, {0, ...|
|          5|            0|[{7, 9, 40}, {7, ...|
|          2|            0|[{1, 8, 68}, {1, ...|
|          6|            0|[{2, 9, 114}, {2,...|
|          6|            0|[{5, 11, 36}, {5,...|
|          3|            0|[{8, 10, 19}, {8,...|
|          6|            0|[{9, 13, 120}, {9...|
|          1|            0|      [{13, 19, 18}]|
|          4|            0|[{14, 15, 77}, {1...|
|          2|            0|[{15, 22, 13}, {1...|
|          7|            0|[{4, 11, 87}, {4,...|
|          6|            0|[{11, 20, 22}, {1...|
|          5|            0|[{12, 15, 24}, {1...|
|          2|            0|[{16, 17, 58}, {1...|
|          3|            0|[{3, 10, 105}, {3...|
|          2|            0|[{6, 9, 62}, {6, ...|
|          1|            0|       [{10, 21, 8}]|
|          1|       

In [168]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [169]:
max_by_group.take(10)

[(0, 7, 87),
 (7, 9, 40),
 (1, 8, 68),
 (2, 9, 114),
 (5, 12, 58),
 (8, 17, 21),
 (9, 13, 120),
 (13, 19, 18),
 (14, 15, 77),
 (15, 16, 23)]

In [170]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0   1    2
0    0   7   87
1    7   9   40
2    1   8   68
3    2   9  114
4    5  12   58
5    8  17   21
6    9  13  120
7   13  19   18
8   14  15   77
9   15  16   23
10   4  11   87
11  11  16   35
12  12  16   27
13  16  17   58
14   3  10  105
15   6   9   62
16  10  21    8
17  17  20   16
18  18  19   40
19  20  22   17
20  19  21    7

In [171]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0     7      87
1     7     9      40
2     1     8      68
3     2     9     114
4     5    12      58

In [172]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0      0     7      87
1      7     9      40
2      1     8      68
3      2     9     114
4      5    12      58
5      8    17      21
6      9    13     120
7     13    19      18
8     14    15      77
9     15    16      23
10     4    11      87
11    11    16      35
12    12    16      27
13    16    17      58
14     3    10     105
15     6     9      62
16    10    21       8
17    17    20      16
18    18    19      40
19    20    22      17
20    19    21       7

In [173]:
# avg_val = n["weight"].mean()
# avg_val = round(avg_val, 0)
# avg_val

# new_df = n[(n['weight'] > avg_val)]

In [174]:
# new_df = n[(n['weight']>=1)]
# new_df["P_id"] = new_df["P_id"] + 1
# new_df["N_id"] = new_df["N_id"] + 1
# new_df

In [175]:
max1 = n["weight"].max()
max1 = round(max1,0)
max1

120

In [176]:
max_percentage = (max1*35)/100
new_df = n[(n['weight']>max_percentage)]
new_df["P_id"] = new_df["P_id"] + 1
new_df["N_id"] = new_df["N_id"] + 1
new_df

/tmp/ipykernel_9181/2286123994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["P_id"] = new_df["P_id"] + 1
/tmp/ipykernel_9181/2286123994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["N_id"] = new_df["N_id"] + 1


P_id  N_id  weight
0      1     8      87
2      2     9      68
3      3    10     114
4      6    13      58
6     10    14     120
8     15    16      77
10     5    12      87
13    17    18      58
14     4    11     105
15     7    10      62

In [177]:
new_df["c_id"] = [x for x in range(0, len(new_df))]
new_df

col = ["P_id", "N_id", "weight", "cluster_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show(20)

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
|   1|   8|    87|         0|
|   2|   9|    68|         1|
|   3|  10|   114|         2|
|   6|  13|    58|         3|
|  10|  14|   120|         4|
|  15|  16|    77|         5|
|   5|  12|    87|         6|
|  17|  18|    58|         7|
|   4|  11|   105|         8|
|   7|  10|    62|         9|
+----+----+------+----------+



/tmp/ipykernel_9181/4287424093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["c_id"] = [x for x in range(0, len(new_df))]


In [178]:
new_df = new_df.sort_values(by=['P_id'])

In [179]:
new_df.head(20)

P_id  N_id  weight  c_id
0      1     8      87     0
2      2     9      68     1
3      3    10     114     2
14     4    11     105     8
10     5    12      87     6
4      6    13      58     3
15     7    10      62     9
6     10    14     120     4
8     15    16      77     5
13    17    18      58     7

In [180]:
# for i in range(len(new_df)):
#     P = new_df[0]
#     N = new_df[1]
    
#     print(P)
    
    

In [181]:
col = ["P_id","N_id","weight","c_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+----+
|P_id|N_id|weight|c_id|
+----+----+------+----+
|   1|   8|    87|   0|
|   2|   9|    68|   1|
|   3|  10|   114|   2|
|   4|  11|   105|   8|
|   5|  12|    87|   6|
|   6|  13|    58|   3|
|   7|  10|    62|   9|
|  10|  14|   120|   4|
|  15|  16|    77|   5|
|  17|  18|    58|   7|
+----+----+------+----+



In [182]:
from pyspark.sql import Window
import pyspark.sql.functions as f

In [183]:
df_weight = df_weight.withColumn('cluster_id', f.first('c_id').over(Window.partitionBy('N_id')))
df_weight.show()

+----+----+------+----+----------+
|P_id|N_id|weight|c_id|cluster_id|
+----+----+------+----+----------+
|   1|   8|    87|   0|         0|
|   2|   9|    68|   1|         1|
|   3|  10|   114|   2|         2|
|   7|  10|    62|   9|         2|
|   4|  11|   105|   8|         8|
|   5|  12|    87|   6|         6|
|   6|  13|    58|   3|         3|
|  10|  14|   120|   4|         4|
|  15|  16|    77|   5|         5|
|  17|  18|    58|   7|         7|
+----+----+------+----+----------+



In [184]:
df_weight_pd = df_weight.toPandas()
df_weight_pd

P_id  N_id  weight  c_id  cluster_id
0     1     8      87     0           0
1     2     9      68     1           1
2     3    10     114     2           2
3     7    10      62     9           2
4     4    11     105     8           8
5     5    12      87     6           6
6     6    13      58     3           3
7    10    14     120     4           4
8    15    16      77     5           5
9    17    18      58     7           7

In [185]:
pid_df = pd.DataFrame(df_weight_pd.filter(["P_id","N_id","weight","cluster_id"]))
pid_df

P_id  N_id  weight  cluster_id
0     1     8      87           0
1     2     9      68           1
2     3    10     114           2
3     7    10      62           2
4     4    11     105           8
5     5    12      87           6
6     6    13      58           3
7    10    14     120           4
8    15    16      77           5
9    17    18      58           7

In [186]:
nid_df = pd.DataFrame(new_df.iloc[:,1])
nid_df = nid_df.reset_index()
nid_df = nid_df.drop(["index"], axis = 1)
nid_df

N_id
0     8
1     9
2    10
3    11
4    12
5    13
6    10
7    14
8    16
9    18

In [187]:
# nid_dict = {}
# for i in range(len(nid_df)):
#     nid_dict[i] = nid_df[i]

In [188]:
df3 = pd.merge(pid_df, nid_df, how='outer', left_on='P_id', right_on='N_id')
df3 = df3.drop_duplicates()
df3 = df3.dropna(thresh=3)
df3 = df3.fillna(0)
df3 = df3.reset_index()
df3 = df3.drop(["index"], axis = 1)
df3

P_id  N_id_x  weight  cluster_id  N_id_y
0   1.0     8.0    87.0         0.0     0.0
1   2.0     9.0    68.0         1.0     0.0
2   3.0    10.0   114.0         2.0     0.0
3   7.0    10.0    62.0         2.0     0.0
4   4.0    11.0   105.0         8.0     0.0
5   5.0    12.0    87.0         6.0     0.0
6   6.0    13.0    58.0         3.0     0.0
7  10.0    14.0   120.0         4.0    10.0
8  15.0    16.0    77.0         5.0     0.0
9  17.0    18.0    58.0         7.0     0.0

In [189]:
lst = [x for x in df3["N_id_x"]]

for i in range(len(df3)):
    a = (df3.loc[i,"P_id"])
    if(a in lst):
        j = lst.index(a)
        df3.loc[i,"cluster_id"] = df3.loc[j,"cluster_id"]
        #print(df3.loc[i,"P_id"],df3.loc[i,"cluster_id"])
        
df_weight = df3.drop(["N_id_y"], axis = 1)
df_weight

P_id  N_id_x  weight  cluster_id
0   1.0     8.0    87.0         0.0
1   2.0     9.0    68.0         1.0
2   3.0    10.0   114.0         2.0
3   7.0    10.0    62.0         2.0
4   4.0    11.0   105.0         8.0
5   5.0    12.0    87.0         6.0
6   6.0    13.0    58.0         3.0
7  10.0    14.0   120.0         2.0
8  15.0    16.0    77.0         5.0
9  17.0    18.0    58.0         7.0

In [190]:
col = ["P_id","N_id","weight","cluster_id"]
df_weight = spark.createDataFrame(df_weight, schema=col)
df_weight.show()

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
| 1.0| 8.0|  87.0|       0.0|
| 2.0| 9.0|  68.0|       1.0|
| 3.0|10.0| 114.0|       2.0|
| 7.0|10.0|  62.0|       2.0|
| 4.0|11.0| 105.0|       8.0|
| 5.0|12.0|  87.0|       6.0|
| 6.0|13.0|  58.0|       3.0|
|10.0|14.0| 120.0|       2.0|
|15.0|16.0|  77.0|       5.0|
|17.0|18.0|  58.0|       7.0|
+----+----+------+----------+



In [191]:
ss = SparkSession(sc)


In [192]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",
                True)\
        .csv('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/sample1_sample2.csv')
df1.show(5)

+---+--------------+--------------+-------------+-----------+--------------------+------+-------------+-------+-------+------+---------------+--------+
| id|          name|    fathername|   mothername|dateofbirth|            location|  city|        state|country|pincode|gender|  qualification|   hobby|
+---+--------------+--------------+-------------+-----------+--------------------+------+-------------+-------+-------+------+---------------+--------+
|  1|  Omkar   jori|    Nathu jori|   Aruna jori| 1999-01-13|             kothrud|  Pune|   Maharastra|  India| 411038|     M|BE in mechnical|swimming|
|  2|      Mr Gupta|       Suresh | Neelam gupta| 1994-03-11|                null|  Orai|Uttar pradesh|  India| 285001|     M| diploma in CSE|  eating|
|  3|Amruta  Argade|Diwakar Argade|Sunita Argade| 1993-09-14| sankul, Near zea...|  Pune|   Maharastra|  India| 411041|     F|           ENTC| dancing|
|  4|    varun jhaa|    Arvind Jha|  Urvashi Jha| 1998-07-15| M-04,Vaishali Nagar|Jaipur

In [193]:
from pyspark.sql.functions import monotonically_increasing_id


In [194]:
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [195]:
pd_common_df = common_df.toPandas()
pd_common_df

id                   name       fathername     mothername dateofbirth  \
0   1           Omkar   jori       Nathu jori     Aruna jori  1999-01-13   
1   2               Mr Gupta          Suresh    Neelam gupta  1994-03-11   
2   3         Amruta  Argade   Diwakar Argade  Sunita Argade  1993-09-14   
3   7            Miss Amruta             None        Sunita         None   
4   4             varun jhaa       Arvind Jha    Urvashi Jha  1998-07-15   
5   5            Aditi Patil     Hemant Patil   Yogita bonde  1999-08-22   
6   6           Bhumik Patil    Patil Hemaant   Lalita bonde  2003-09-06   
7  10  Argade Amruta Diwakar   Diwakar Argade  Sunita Argade  1993-09-14   
8  15          Amruta Argade      Anad Argade  Shweta Argade  2003-08-16   
9  17         Amruta Bhosale  Pradeep Bhosale   Lata Bhosale  1999-06-19   

                                            location               city  \
0                                            kothrud               Pune   
1                                               None               Orai   
2              sankul, Near zeal engineering college               Pune   
3              sankul, Near zeal engineering college  Panchavati Pashan   
4                                M-04,Vaishali Nagar             Jaipur   
5                                      khanda colony             Panvel   
6                                        Sai darshan          NewPanvel   
7  Keshav rukmini sankul, Near zeal engineering c...               Pune   
8                                      Anand Society            Solapur   
9                                  Sai Krupa housing             Panvel   

           state country   pincode gender    qualification     hobby  P_id  \
0     Maharastra   India  411038.0      M  BE in mechnical  swimming   1.0   
1  Uttar pradesh   India  285001.0      M   diploma in CSE    eating   2.0   
2     Maharastra   India  411041.0      F             ENTC   dancing   3.0   
3           None   India       NaN      F             ENTC   dancing   7.0   
4     Rajasthan    India  302012.0      M        BE in CSE  sleeping   4.0   
5     Maharastra   India  410206.0      F       BE in EXTC    eating   5.0   
6    Maharashtra   India  400026.0      M   Btech EXTC NIT  irritate   6.0   
7     Maharastra   India  411041.0      F       BE in ENTC   dancing  10.0   
8    Maharashtra   India  420036.0      F              CSE   Cooking  15.0   
9   Maharashtra    India  410206.0      F            BE IT   Cycling  17.0   

   N_id  weight  cluster_id  
0   8.0    87.0         0.0  
1   9.0    68.0         1.0  
2  10.0   114.0         2.0  
3  10.0    62.0         2.0  
4  11.0   105.0         8.0  
5  12.0    87.0         6.0  
6  13.0    58.0         3.0  
7  14.0   120.0         2.0  
8  16.0    77.0         5.0  
9  18.0    58.0         7.0

In [196]:
common_N = df1.join(df_weight, df1.id == df_weight.N_id, "inner")

In [197]:
pd_common_N = common_N.toPandas()
pd_common_N

id                   name            fathername     mothername dateofbirth  \
0   8      Omkar  nathu jori            Nathu jori     Aruna jori  1999-01-13   
1   9            Rohit gupta  Suresh chandra gupta   Neelam gupta  1994-03-11   
2  10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade  1993-09-14   
3  10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade  1993-09-14   
4  11              varun jha            Arvind Jha    Urvashi Jha  1998-07-15   
5  12            Patil Aditi          Hemant Patil   Yogita Patil  1999-08-22   
6  13   Bhumika Hemant Patil          Hemant Patil           None  2000-09-16   
7  14           Mrs Deshmukh        Diwakar Argade  Sunita Argade  1993-09-14   
8  16    Amruta Anand Argade           Anad Argade  Shweta Argade  2003-08-16   
9  18     Ms Amruta Lokhande       Pradeep Bhosale   Lata Bhosale  1999-06-19   

                                            location         city  \
0                        Kalashree MD Rivera Bhugaon         Pune   
1                                        tulsi nagar         Orai   
2  Keshav rukmini sankul, Near zeal engineering c...         Pune   
3  Keshav rukmini sankul, Near zeal engineering c...         Pune   
4               M-04,Ashiana Mangalam,Vaishali Nagar       Jaipur   
5                                     Gurudev Arcade       Panvel   
6                                        Sai darshan  Navi Mumbai   
7  Keshav rukmini sankul, Near zeal engineering c...        Baner   
8                                       Gandhi nagar      Solapur   
9                           Sector 10, Neelam Arcade    Hyderabad   

           state country  pincode gender     qualification             hobby  \
0     Maharastra   India   411041      M   BE in mechnical          swimming   
1  Uttar pradesh   India   285001      M         BE in ECE            runing   
2     Maharastra   India   411041      F        BE in ENTC           dancing   
3     Maharastra   India   411041      F        BE in ENTC           dancing   
4     Rajasthan    India   302012      M         BE in CSE          sleeping   
5     Maharastra   India   410206      F        BE in EXTC            eating   
6    Maharashtra   India   400046      F             Btech          sleeping   
7     Maharastra   India   411041      F        BE in ENTC           dancing   
8    Maharashtra   India   420036      F  Computer science           Cooking   
9     Telangana    India   512007      F             BE IT  Cycling Teaching   

   P_id  N_id  weight  cluster_id  
0   1.0   8.0    87.0         0.0  
1   2.0   9.0    68.0         1.0  
2   3.0  10.0   114.0         2.0  
3   7.0  10.0    62.0         2.0  
4   4.0  11.0   105.0         8.0  
5   5.0  12.0    87.0         6.0  
6   6.0  13.0    58.0         3.0  
7  10.0  14.0   120.0         2.0  
8  15.0  16.0    77.0         5.0  
9  17.0  18.0    58.0         7.0

In [198]:
import functools


def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)


# unionAll
result3 = unionAll([common_df, common_N])
result3.show()

final = result3.toPandas()

final = final.drop(["P_id", "N_id"], axis=1)
final.head()

final_cluster = final.groupby(['cluster_id', 'id'])
final_cluster.first()

+---+--------------------+--------------------+-------------+-----------+--------------------+-----------------+-------------+-------+-------+------+----------------+----------------+----+----+------+----------+
| id|                name|          fathername|   mothername|dateofbirth|            location|             city|        state|country|pincode|gender|   qualification|           hobby|P_id|N_id|weight|cluster_id|
+---+--------------------+--------------------+-------------+-----------+--------------------+-----------------+-------------+-------+-------+------+----------------+----------------+----+----+------+----------+
|  1|        Omkar   jori|          Nathu jori|   Aruna jori| 1999-01-13|             kothrud|             Pune|   Maharastra|  India| 411038|     M| BE in mechnical|        swimming| 1.0| 8.0|  87.0|       0.0|
|  2|            Mr Gupta|             Suresh | Neelam gupta| 1994-03-11|                null|             Orai|Uttar pradesh|  India| 285001|     M|  d

name            fathername     mothername  \
cluster_id id                                                               
0.0        1            Omkar   jori            Nathu jori     Aruna jori   
           8       Omkar  nathu jori            Nathu jori     Aruna jori   
1.0        2                Mr Gupta               Suresh    Neelam gupta   
           9             Rohit gupta  Suresh chandra gupta   Neelam gupta   
2.0        3          Amruta  Argade        Diwakar Argade  Sunita Argade   
           7             Miss Amruta                  None        Sunita    
           10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
           14           Mrs Deshmukh        Diwakar Argade  Sunita Argade   
3.0        6            Bhumik Patil         Patil Hemaant   Lalita bonde   
           13   Bhumika Hemant Patil          Hemant Patil           None   
5.0        15          Amruta Argade           Anad Argade  Shweta Argade   
           16    Amruta Anand Argade           Anad Argade  Shweta Argade   
6.0        5             Aditi Patil          Hemant Patil   Yogita bonde   
           12            Patil Aditi          Hemant Patil   Yogita Patil   
7.0        17         Amruta Bhosale       Pradeep Bhosale   Lata Bhosale   
           18     Ms Amruta Lokhande       Pradeep Bhosale   Lata Bhosale   
8.0        4              varun jhaa            Arvind Jha    Urvashi Jha   
           11              varun jha            Arvind Jha    Urvashi Jha   

              dateofbirth                                           location  \
cluster_id id                                                                  
0.0        1   1999-01-13                                            kothrud   
           8   1999-01-13                        Kalashree MD Rivera Bhugaon   
1.0        2   1994-03-11                                               None   
           9   1994-03-11                                        tulsi nagar   
2.0        3   1993-09-14              sankul, Near zeal engineering college   
           7         None              sankul, Near zeal engineering college   
           10  1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
           14  1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
3.0        6   2003-09-06                                        Sai darshan   
           13  2000-09-16                                        Sai darshan   
5.0        15  2003-08-16                                      Anand Society   
           16  2003-08-16                                       Gandhi nagar   
6.0        5   1999-08-22                                      khanda colony   
           12  1999-08-22                                     Gurudev Arcade   
7.0        17  1999-06-19                                  Sai Krupa housing   
           18  1999-06-19                           Sector 10, Neelam Arcade   
8.0        4   1998-07-15                                M-04,Vaishali Nagar   
           11  1998-07-15               M-04,Ashiana Mangalam,Vaishali Nagar   

                            city          state country   pincode gender  \
cluster_id id                                                              
0.0        1                Pune     Maharastra   India  411038.0      M   
           8                Pune     Maharastra   India  411041.0      M   
1.0        2                Orai  Uttar pradesh   India  285001.0      M   
           9                Orai  Uttar pradesh   India  285001.0      M   
2.0        3                Pune     Maharastra   India  411041.0      F   
           7   Panchavati Pashan           None   India       NaN      F   
           10               Pune     Maharastra   India  411041.0      F   
           14              Baner     Maharastra   India  411041.0      F   
3.0        6           NewPanvel    Maharashtra   India  400026.0      M   
           13        Navi Mumbai    Maharashtra   Indi

In [199]:
pd_common_df = pd_common_df.sort_values(by=['P_id'])
pd_common_df

id                   name       fathername     mothername dateofbirth  \
0   1           Omkar   jori       Nathu jori     Aruna jori  1999-01-13   
1   2               Mr Gupta          Suresh    Neelam gupta  1994-03-11   
2   3         Amruta  Argade   Diwakar Argade  Sunita Argade  1993-09-14   
4   4             varun jhaa       Arvind Jha    Urvashi Jha  1998-07-15   
5   5            Aditi Patil     Hemant Patil   Yogita bonde  1999-08-22   
6   6           Bhumik Patil    Patil Hemaant   Lalita bonde  2003-09-06   
3   7            Miss Amruta             None        Sunita         None   
7  10  Argade Amruta Diwakar   Diwakar Argade  Sunita Argade  1993-09-14   
8  15          Amruta Argade      Anad Argade  Shweta Argade  2003-08-16   
9  17         Amruta Bhosale  Pradeep Bhosale   Lata Bhosale  1999-06-19   

                                            location               city  \
0                                            kothrud               Pune   
1                                               None               Orai   
2              sankul, Near zeal engineering college               Pune   
4                                M-04,Vaishali Nagar             Jaipur   
5                                      khanda colony             Panvel   
6                                        Sai darshan          NewPanvel   
3              sankul, Near zeal engineering college  Panchavati Pashan   
7  Keshav rukmini sankul, Near zeal engineering c...               Pune   
8                                      Anand Society            Solapur   
9                                  Sai Krupa housing             Panvel   

           state country   pincode gender    qualification     hobby  P_id  \
0     Maharastra   India  411038.0      M  BE in mechnical  swimming   1.0   
1  Uttar pradesh   India  285001.0      M   diploma in CSE    eating   2.0   
2     Maharastra   India  411041.0      F             ENTC   dancing   3.0   
4     Rajasthan    India  302012.0      M        BE in CSE  sleeping   4.0   
5     Maharastra   India  410206.0      F       BE in EXTC    eating   5.0   
6    Maharashtra   India  400026.0      M   Btech EXTC NIT  irritate   6.0   
3           None   India       NaN      F             ENTC   dancing   7.0   
7     Maharastra   India  411041.0      F       BE in ENTC   dancing  10.0   
8    Maharashtra   India  420036.0      F              CSE   Cooking  15.0   
9   Maharashtra    India  410206.0      F            BE IT   Cycling  17.0   

   N_id  weight  cluster_id  
0   8.0    87.0         0.0  
1   9.0    68.0         1.0  
2  10.0   114.0         2.0  
4  11.0   105.0         8.0  
5  12.0    87.0         6.0  
6  13.0    58.0         3.0  
3  10.0    62.0         2.0  
7  14.0   120.0         2.0  
8  16.0    77.0         5.0  
9  18.0    58.0         7.0

In [200]:
# pd_common_df = pd_common_df[~pd_common_df.id.isin(nid_df.N_id)]
# pd_common_df

In [201]:
pd_common_df = pd_common_df.sort_values(by=['id'])
pd_common_df

id                   name       fathername     mothername dateofbirth  \
0   1           Omkar   jori       Nathu jori     Aruna jori  1999-01-13   
1   2               Mr Gupta          Suresh    Neelam gupta  1994-03-11   
2   3         Amruta  Argade   Diwakar Argade  Sunita Argade  1993-09-14   
4   4             varun jhaa       Arvind Jha    Urvashi Jha  1998-07-15   
5   5            Aditi Patil     Hemant Patil   Yogita bonde  1999-08-22   
6   6           Bhumik Patil    Patil Hemaant   Lalita bonde  2003-09-06   
3   7            Miss Amruta             None        Sunita         None   
7  10  Argade Amruta Diwakar   Diwakar Argade  Sunita Argade  1993-09-14   
8  15          Amruta Argade      Anad Argade  Shweta Argade  2003-08-16   
9  17         Amruta Bhosale  Pradeep Bhosale   Lata Bhosale  1999-06-19   

                                            location               city  \
0                                            kothrud               Pune   
1                                               None               Orai   
2              sankul, Near zeal engineering college               Pune   
4                                M-04,Vaishali Nagar             Jaipur   
5                                      khanda colony             Panvel   
6                                        Sai darshan          NewPanvel   
3              sankul, Near zeal engineering college  Panchavati Pashan   
7  Keshav rukmini sankul, Near zeal engineering c...               Pune   
8                                      Anand Society            Solapur   
9                                  Sai Krupa housing             Panvel   

           state country   pincode gender    qualification     hobby  P_id  \
0     Maharastra   India  411038.0      M  BE in mechnical  swimming   1.0   
1  Uttar pradesh   India  285001.0      M   diploma in CSE    eating   2.0   
2     Maharastra   India  411041.0      F             ENTC   dancing   3.0   
4     Rajasthan    India  302012.0      M        BE in CSE  sleeping   4.0   
5     Maharastra   India  410206.0      F       BE in EXTC    eating   5.0   
6    Maharashtra   India  400026.0      M   Btech EXTC NIT  irritate   6.0   
3           None   India       NaN      F             ENTC   dancing   7.0   
7     Maharastra   India  411041.0      F       BE in ENTC   dancing  10.0   
8    Maharashtra   India  420036.0      F              CSE   Cooking  15.0   
9   Maharashtra    India  410206.0      F            BE IT   Cycling  17.0   

   N_id  weight  cluster_id  
0   8.0    87.0         0.0  
1   9.0    68.0         1.0  
2  10.0   114.0         2.0  
4  11.0   105.0         8.0  
5  12.0    87.0         6.0  
6  13.0    58.0         3.0  
3  10.0    62.0         2.0  
7  14.0   120.0         2.0  
8  16.0    77.0         5.0  
9  18.0    58.0         7.0

In [202]:
# pd_common_df.reset_index(inplace = True)
# pd_common_df = pd_common_df.drop(["index"], axis=1)

In [203]:
# pd_common_df = pd_common_df.drop(["identifiers_right","keyValuePairs_right","P_id","N_id","weight"], axis =1)
# pd_common_df

In [204]:
# pd_common_df["new_id"] = [x for x in range(0,len(pd_common_df))]
# pd_common_df = pd_common_df.drop(["id"], axis=1)

In [205]:
# sp_df2 = spark.createDataFrame(pd_common_df)
# sp_df2

In [206]:
# pd_common_df.to_json('/home/aditi/BigData/AAI_Project/dataset/df2_F2.json')

In [207]:
# baseDir = 'hdfs://localhost:9000/Final_Project/sample/DF_op/Df2_F2.json'
# sp_df2.write.option("header", True).mode('overwrite').json(baseDir)